Pandas GroupBy
==============

This is a Jupyter notebook I make for learning Pandas. Material is from [https://realpython.com/pandas-groupby/](https://realpython.com/pandas-groupby/).

In [1]:
%matplotlib inline

import pandas as pd

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
# pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
# pd.set_option("display.max_rows", 25)

In [2]:
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df = pd.read_csv(
    "groupby-data/legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)
df.tail()

,last_name,first_name,birthday,gender,type,state,party
11970,Garrett,Thomas,1972-03-27,M,rep,VA,Republican
11971,Handel,Karen,1962-04-18,F,rep,GA,Republican
11972,Jones,Brenda,1959-10-24,F,rep,MI,Democrat
11973,Marino,Tom,1952-08-15,M,rep,PA,Republican
11974,Jones,Walter,1943-02-10,M,rep,NC,Republican


The `DataFrame` uses categorical **dtypes** for space efficiency

In [3]:
df.dtypes

last_name             object
first_name          category
birthday      datetime64[ns]
gender              category
type                category
state               category
party               category
dtype: object

In [4]:
# Near-equivalent to SELECT state, COUNT(name) FROM df GROUP BY state ORDER BY state;
n_by_state = df.groupby("state")["last_name"].count()
n_by_state.head(10)

state
AK     16
AL    206
AR    117
AS      2
AZ     48
CA    361
CO     90
CT    240
DC      2
DE     97
Name: last_name, dtype: int64

In [5]:
# Near-equivalent to SELECT state, gender, COUNT(name) FROM df GROUP BY state, gender ORDER BY state, gender;
n_by_state_gender = df.groupby(["state", "gender"])["last_name"].count()
n_by_state_gender

state  gender
AK     F           0
       M          16
AL     F           3
       M         203
AR     F           5
                ... 
WI     M         196
WV     F           1
       M         119
WY     F           2
       M          38
Name: last_name, Length: 116, dtype: int64

In [6]:
type(n_by_state_gender)

pandas.core.series.Series

In [7]:
n_by_state_gender.index[:5]

MultiIndex([('AK', 'F'),
            ('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F')],
           names=['state', 'gender'])

Difference between the SQL result and above result is the SQL result contains 3 columns: `state`, `gender`, and `count`. The following more closely emulate the SQL result:

In [8]:
df.groupby(["state", "gender"], as_index=False)["last_name"].count()

,state,gender,last_name
0,AK,F,NaN
1,AK,M,16.0
2,AL,F,3.0
3,AL,M,203.0
4,AR,F,5.0
...,...,...,...
111,WI,M,196.0
112,WV,F,1.0
113,WV,M,119.0
114,WY,F,2.0


In [9]:
# if you don't want automatic sort
df.groupby("state", sort=False)["last_name"].count()

state
DE      97
VA     432
SC     251
MD     305
PA    1053
MA     426
NJ     359
GA     309
NY    1461
NC     354
CT     240
VT     115
KY     373
RI     107
NH     181
TN     299
OH     674
MS     155
OL       2
IN     341
LA     197
IL     486
MO     333
AL     206
AR     117
ME     175
FL     155
MI     294
IA     202
WI     196
TX     256
CA     361
OR      89
MN     160
NM      54
NE     127
WA      95
KS     141
UT      53
NV      56
CO      90
WV     120
DK       9
AZ      48
ID      59
MT      52
WY      40
DC       2
ND      44
SD      51
OK      92
HI      23
PR      19
AK      16
PI      13
VI       4
GU       4
AS       2
Name: last_name, dtype: int64

In [10]:
by_state = df.groupby("state")
by_state

The above shows that pandas groupby is lazy in nature. It doesn’t really do any operations to produce a useful result until you say so.

In [11]:
show_n_entries = 2
for state, frame in by_state:
    print(f"First {show_n_entries} entries for {state!r}")
    print("------------------------")
    print(frame.head(show_n_entries), end="\n\n")

First 2 entries for 'AK'
------------------------
     last_name first_name   birthday gender type state        party
6619    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6647      Cale     Thomas 1848-09-17      M  rep    AK  Independent

First 2 entries for 'AL'
------------------------
    last_name first_name   birthday gender type state       party
912   Crowell       John 1780-09-18      M  rep    AL  Republican
991    Walker       John 1783-08-12      M  sen    AL  Republican

First 2 entries for 'AR'
------------------------
     last_name first_name   birthday gender type state party
1001     Bates      James 1788-08-25      M  rep    AR   NaN
1279    Conway      Henry 1793-03-18      M  rep    AR   NaN

First 2 entries for 'AS'
------------------------
          last_name first_name   birthday gender type state     party
10797         Sunia       Fofó 1937-03-13      M  rep    AS  Democrat
11755  Faleomavaega        Eni 1943-08-15      M  rep    AS  Democrat

F

In [12]:
by_state.groups["PA"]

Int64Index([    4,    19,    21,    27,    38,    57,    69,    76,    84,
               88,
            ...
            11842, 11866, 11875, 11877, 11887, 11891, 11932, 11945, 11959,
            11973],
           dtype='int64', length=1053)

In [13]:
by_state.get_group("PA")

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11891,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11932,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11945,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11959,Costello,Ryan,1976-09-07,M,rep,PA,Republican


In [14]:
# same as above
df.loc[df["state"] == "PA"]

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11891,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11932,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11945,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11959,Costello,Ryan,1976-09-07,M,rep,PA,Republican


In [15]:
state, frame = next(iter(by_state))
state

'AK'

In [16]:
frame.head(3)

,last_name,first_name,birthday,gender,type,state,party
6619,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6647,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7442,Grigsby,George,1874-12-02,M,rep,AK,NaN


In [17]:
frame["last_name"].count()

16

In [18]:
df = pd.read_csv(
    "groupby-data/airqual.csv",
    parse_dates=[["Date", "Time"]],
    na_values=[-200],
    usecols=["Date", "Time", "CO(GT)", "T", "RH", "AH"]
).rename(
    columns={
        "CO(GT)": "co",
        "Date_Time": "tstamp",
        "T": "temp_c",
        "RH": "rel_hum",
        "AH": "abs_hum",
    }
).set_index("tstamp")
df.head()

,co,temp_c,rel_hum,abs_hum
tstamp,,,,
2004-03-10 18:00:00,2.6,13.6,48.9,0.758
2004-03-10 19:00:00,2.0,13.3,47.7,0.726
2004-03-10 20:00:00,2.2,11.9,54.0,0.750
2004-03-10 21:00:00,2.2,11.0,60.0,0.787
2004-03-10 22:00:00,1.6,11.2,59.6,0.789
